In [1]:
import re
import json
import os
from typing import Optional, List, Any, Dict, Set, Tuple
import numpy as np
from copy import deepcopy
import random

In [2]:
def read_json(fname: str) -> Any:
    """
    Given a filename, reads a json file and returns the data stored inside.

    Input:
        fname (str):
            Name of the file to be read.

    Output:
        data (Any):
            The data loaded from the json file.
    """

    assert os.path.isfile(fname)
    assert fname.endswith(".json")

    with open(fname, "r") as file:
        data = json.load(file)

    return data


def write_json(
    data: Any,
    fname: str,
) -> None:
    """
    Given a data and the filename, writes the data to the specified
    fname.
    If the directory that the specified filename should be in
    does not exist, then it creates the directory first.

    Input:
        data (Any):
            the data that needs to stored in a json format.

        fname (str):
            path to the file where the data needs to be saved.

    Output:
        None
    """

    assert isinstance(fname, str) and fname.endswith(".json")
    splits = fname.split("/")[:-1]
    root_dir = "/".join(splits)
    if not os.path.isdir(root_dir):
        os.makedirs(root_dir, exist_ok=True)

    with open(fname, "w") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [3]:
def place_mines_excluding_first_agent_move(
    safe_r: int,
    safe_c: int,
    total_mines: int,
    rows: int,
    cols: int,
) -> List[Tuple[int, int]]:
    all_cells = [
        (r, c)
        for r in range(rows)
        for c in range(cols)
    ]

    all_cells.remove((safe_r, safe_c))
    mine_cells = random.sample(
        all_cells,
        min(total_mines, len(all_cells))
    )

    mine_positions = []
    for cell in mine_cells:
        mine_positions.append(list(cell))

    return mine_positions

In [4]:
def render_board_message(
    rows: int,
    cols: int,
) -> str:
    lines = []

    for r in range(rows):
        row_chars = []
        for c in range(cols):
            row_chars.append("#")
        
        lines.append(" ".join(row_chars))

    return "\n".join(lines)


def generate_train_data(
    num_examples: int,
) -> List[Dict[str, Any]]:
    all_scenarios = []
    
    for _ in range(num_examples):
        num_rows = random.randint(4, 6)
        num_cols = random.randint(4, 6)
        
        all_cells = [
            (r, c)
            for r in range(num_rows)
            for c in range(num_cols)
        ]
        
        safe_cell_index = np.random.choice(
            a=len(all_cells),
            size=None,
        )
        safe_cell = all_cells[safe_cell_index]
        all_cells.remove(safe_cell)
        
        suggested = max(1, (num_rows * num_cols) // 10)
        random_mine_count = min(suggested, num_rows * num_cols - 1)
        
        mine_cells = random.sample(
            all_cells,
            min(random_mine_count, len(all_cells))
        )
        
        mine_positions = []
        for mine_cell in mine_cells:
            mine_positions.append(list(mine_cell))
            
        scenario = {
            "env": {
                "mine_positions": mine_positions,
                "rows": num_rows,
                "cols": num_cols,
            },
            "agent": {
                "first_agent_move": list(safe_cell),
                "initial_board_view": render_board_message(
                    rows=num_rows,
                    cols=num_cols,
                ),
            }
        }
        
        all_scenarios.append(scenario)
        
    return all_scenarios


def generate_eval_data(
    num_examples: int,
) -> List[Dict[str, Any]]:
    all_scenarios = []
    
    for _ in range(num_examples):
        rand_num = random.randint(0, 1)
        if rand_num == 0:
            num_rows = 7
            num_cols = random.randint(4, 7)
            
        else:
            num_rows = random.randint(4, 7)
            num_rows = 7
        
        suggested = max(1, (num_rows * num_cols) // 10)
        random_mine_count = min(suggested, num_rows * num_cols - 1)
        
        scenario = {
            "env": {
                "rows": num_rows,
                "cols": num_cols,
                "random_mine_count": random_mine_count,
            },
            "agent": {
                "initial_board_view": render_board_message(
                    rows=num_rows,
                    cols=num_cols,
                ),
            }
        }
        
        all_scenarios.append(scenario)
        
    return all_scenarios

In [5]:
def generate_minesweeper_config_file(
    path_to_old_config: str,
    path_to_new_config: str,
    num_train_examples: int,
    num_test_examples: int,
) -> None:
    data = read_json(
        fname=path_to_old_config,
    )
    
    data["train"] = generate_train_data(
        num_examples=num_train_examples,
    )
    
    data["eval"] = generate_eval_data(
        num_examples=num_test_examples,
    )
    
    write_json(
        data=data,
        fname=path_to_new_config,
    )

In [6]:
generate_minesweeper_config_file(
    path_to_old_config="/Users/fahimtajwar/academics/exploration/verl/verl/paprika/environments/env_configs/minesweeper.json",
    path_to_new_config="/Users/fahimtajwar/academics/exploration/verl/verl/paprika/environments/env_configs/minesweeper_new.json",
    num_train_examples=1000,
    num_test_examples=1000,
)